In [2]:
import pandas as pd
import numpy as np
import gmplot

import matplotlib.pyplot as plt

import folium
from IPython.display import display

import sklearn
import sklearn.preprocessing as pre
import sklearn.model_selection as mdl

In [3]:
accident1 = pd.read_csv('../RawData/KSI(1999-2014).csv')
accident2 = pd.read_csv('../RawData/KSI(2008-2018).csv')

In [4]:
accident = accident2[['DATE','LATITUDE', 'LONGITUDE', 'STREET1', 'STREET2', 'ROAD_CLASS', 'District', 
                      'LOCCOORD', 'ACCLOC', 'INJURY', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 
                      'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL']]

In [5]:
accident.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12244 entries, 0 to 12243
Data columns (total 18 columns):
DATE          12244 non-null object
LATITUDE      12244 non-null float64
LONGITUDE     12244 non-null float64
STREET1       12244 non-null object
STREET2       12244 non-null object
ROAD_CLASS    12244 non-null object
District      12244 non-null object
LOCCOORD      12244 non-null object
ACCLOC        12244 non-null object
INJURY        12244 non-null object
INITDIR       12244 non-null object
VEHTYPE       12244 non-null object
MANOEUVER     12244 non-null object
DRIVACT       12244 non-null object
SPEEDING      12244 non-null object
AG_DRIV       12244 non-null object
REDLIGHT      12244 non-null object
ALCOHOL       12244 non-null object
dtypes: float64(2), object(16)
memory usage: 1.7+ MB


In [6]:
accident.head()

,DATE,LATITUDE,LONGITUDE,STREET1,STREET2,ROAD_CLASS,District,LOCCOORD,ACCLOC,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL
0,2017-06-03T04:00:00.000Z,43.675280,-79.454730,ST CLAIR AVE W,CALEDONIA RD,Major Arterial,Toronto and East York,Intersection,Intersection Related,Major,West,Bicycle,Going Ahead,,,Yes,,
1,2017-09-25T04:00:00.000Z,43.770049,-79.520641,4673 JANE ST,,Major Arterial,Etobicoke York,Mid-Block,Non Intersection,None,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Yes,Yes,,
2,2017-09-25T04:00:00.000Z,43.770049,-79.520641,4673 JANE ST,,Major Arterial,Etobicoke York,Mid-Block,Non Intersection,Major,East,,,,Yes,Yes,,
3,2017-12-01T05:00:00.000Z,43.751868,-79.286601,BIRCHMOUNT RD,MERRYFIELD DR,Major Arterial,Scarborough,Intersection,At Intersection,None,North,Passenger Van,Going Ahead,Driving Properly,,,,
4,2017-12-01T05:00:00.000Z,43.751868,-79.286601,BIRCHMOUNT RD,MERRYFIELD DR,Major Arterial,Scarborough,Intersection,At Intersection,Fatal,East,,,,,,,


In [7]:
accident["DATE"] = pd.to_datetime(accident["DATE"])
accident.index = accident["DATE"]

C:\Users\Tanvy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
pd.unique(accident['DATE'].dt.year)

array([2017, 2008, 2014, 2015, 2018, 2013, 2009, 2010, 2012, 2016, 2011], dtype=int64)

In [9]:
accident.head()

,DATE,LATITUDE,LONGITUDE,STREET1,STREET2,ROAD_CLASS,District,LOCCOORD,ACCLOC,INJURY,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL
DATE,,,,,,,,,,,,,,,,,,
2017-06-03 04:00:00,2017-06-03 04:00:00,43.675280,-79.454730,ST CLAIR AVE W,CALEDONIA RD,Major Arterial,Toronto and East York,Intersection,Intersection Related,Major,West,Bicycle,Going Ahead,,,Yes,,
2017-09-25 04:00:00,2017-09-25 04:00:00,43.770049,-79.520641,4673 JANE ST,,Major Arterial,Etobicoke York,Mid-Block,Non Intersection,None,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Yes,Yes,,
2017-09-25 04:00:00,2017-09-25 04:00:00,43.770049,-79.520641,4673 JANE ST,,Major Arterial,Etobicoke York,Mid-Block,Non Intersection,Major,East,,,,Yes,Yes,,
2017-12-01 05:00:00,2017-12-01 05:00:00,43.751868,-79.286601,BIRCHMOUNT RD,MERRYFIELD DR,Major Arterial,Scarborough,Intersection,At Intersection,None,North,Passenger Van,Going Ahead,Driving Properly,,,,
2017-12-01 05:00:00,2017-12-01 05:00:00,43.751868,-79.286601,BIRCHMOUNT RD,MERRYFIELD DR,Major Arterial,Scarborough,Intersection,At Intersection,Fatal,East,,,,,,,


In [10]:
accident = accident.replace(' ', np.NAN, regex=False)

In [11]:
accident.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12244 entries, 2017-06-03 04:00:00 to 2011-06-26 04:00:00
Data columns (total 18 columns):
DATE          12244 non-null datetime64[ns]
LATITUDE      12244 non-null float64
LONGITUDE     12244 non-null float64
STREET1       12244 non-null object
STREET2       11191 non-null object
ROAD_CLASS    12223 non-null object
District      12222 non-null object
LOCCOORD      12169 non-null object
ACCLOC        8348 non-null object
INJURY        10915 non-null object
INITDIR       8821 non-null object
VEHTYPE       10095 non-null object
MANOEUVER     7022 non-null object
DRIVACT       6180 non-null object
SPEEDING      2035 non-null object
AG_DRIV       6324 non-null object
REDLIGHT      959 non-null object
ALCOHOL       487 non-null object
dtypes: datetime64[ns](1), float64(2), object(15)
memory usage: 1.8+ MB


In [12]:
accident.to_csv("RouteSaferDataSets.csv")

numpy.ndarray

In [108]:
oneHotEncoder        = pre.OneHotEncoder()
oneHotEncoder.fit(accident_value[:, 6])
accident_value[:, 6] = oneHotEncoder.transform(accident[:, 6])
print(accident)

ValueError: could not convert string to float: 'North York'

In [109]:
cat_features = ['color', 'director_name', 'actor_2_name']
enc = preprocessing.LabelEncoder()
enc.fit(cat_features)
new_cat_features = enc.transform(cat_features)
print new_cat_features # [1 2 0]
new_cat_features = new_cat_features.reshape(-1, 1) # Needs to be the correct shape
ohe = preprocessing.OneHotEncoder(sparse=False) #Easier to read
print ohe.fit_transform(new_cat_features)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(new_cat_features # [1 2 0])? (<ipython-input-109-537919816f25>, line 5)

In [57]:
a = np.array([["qw a","1"], ["wq a","2"], ["qw a","3"], ["qw b","4"]])

In [72]:
a

array([['qw a', '1'],
       ['wq a', '2'],
       ['qw a', '3'],
       ['qw b', '4']],
      dtype='<U4')

In [51]:
l = pre.LabelEncoder()
a[:, 0] = l.fit_transform(a[:, 0])

In [52]:
a

array([['0', '1'],
       ['2', '2'],
       ['0', '3'],
       ['1', '4']],
      dtype='<U4')